# Guide to Unsupervised Pretraining
In this guide, you'll learn how to train a backbone without using labels. At the end of this guide, you'll build a supervised k-nearest-neighbors classifier head on top of this backbone. Although the KNN classifier demonstrates that the backbone has learned representations of the data, see the "Training with a backbone" guide for the recommended applications of a backbone (e.g. linear or MLP head for classification, pyramid feature network for detection, unet for segmentation).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/masterfulai/masterful-docs/blob/main/tutorial_quickstart.ipynb)\
[![download button](_static/download_logo_32px.png)](https://masterful-public.s3.us-west-1.amazonaws.com/933013963/latest/tutorial_quickstart.ipynb) Download this Notebook

First, import necessary libraries:

In [1]:
import numpy as np
import tensorflow as tf

MASTERFUL_ACCOUNT_ID="<INSERT HERE>"
MASTERFUL_AUTHORIZATION_KEY="<INSERT HERE>"
import masterful
masterful = masterful.register(account_id=MASTERFUL_ACCOUNT_ID, authorization_key=MASTERFUL_AUTHORIZATION_KEY)

import masterful.core

MASTERFUL: Your account has been successfully registered. Masterful v0.3.6 is loaded.


For this guide you'll be using the CIFAR10 dataset, which consists of 60,000 color images of 10 separate, non-overlapping classes of objects. There are 6,000 images of each class of object; 5,000 in a training set and 1,000 held out for testing.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

Create a Data Specification, which captures CIFAR10's relevant metadata:

In [3]:
data_spec = masterful.DataSpec.from_dataset(
        task=masterful.spec.Task.CLASSIFICATION,
        dataset=train_dataset,
        image_range=masterful.spec.ImageRange.ZERO_255,
        num_classes=10,
        sparse=False)

Now create a Pretraining Policy, which establishes pretraining-related hyperparameters:

In [4]:
pretrain_policy = masterful.PretrainPolicy(batch_size=512,
                                           epochs=3,
                                           warmup_steps=10)

Finally, create a model to be pretrained – here, the same simple CNN used in [this](https://www.tensorflow.org/tutorials/images/cnn) TensorFlow tutorial for labeled training:

In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

Now you're ready to pretrain! Pretraining with Masterful will return a training report, including fields like
- `loss`: the unsupervised loss at the end of pretraining.
- `accuracy`: the accuracy achieved by performing K-Nearest Neighbors classification on the pretrained backbone's output features

(Note that the labels in the dataset passed to the `labeled_data` parameter below are only used for KNN classification, _not_ for pretraining the backbone.)

In [6]:
training_report = masterful.core.pretrain(
        pretrain_policy=pretrain_policy,
        model=model,
        validation_data=validation_dataset,
        labeled_data=train_dataset,
        unlabeled_data=None,
        synthetic_data=None,
        data_spec=data_spec,
    )

Epoch 1/3
98/98 [==============================] - 155s 2s/step - loss: 1155.8315


Test Epoch: Acc@1:34.23%: 100%|██████████| 20/20 [00:00<00:00, 34.41it/s]


kNN Test Accuracy at epoch 0: 34.22999954223633 Max Accuracy so far: 34.22999954223633
Epoch 2/3
98/98 [==============================] - 153s 2s/step - loss: 967.6478


Test Epoch: Acc@1:36.54%: 100%|██████████| 20/20 [00:00<00:00, 37.89it/s]


kNN Test Accuracy at epoch 1: 36.53999710083008 Max Accuracy so far: 36.53999710083008
Epoch 3/3
98/98 [==============================] - 153s 2s/step - loss: 907.8960


Test Epoch: Acc@1:36.95%: 100%|██████████| 20/20 [00:00<00:00, 37.59it/s]

kNN Test Accuracy at epoch 2: 36.95000076293945 Max Accuracy so far: 36.95000076293945
KnnEvaluator: Restoring model weights from epoch 3 with accuracy 36.95000076293945.


The pretraining policy above only runs for a few epochs, to save time; you should expect better results with more epochs (and quicker results for a smaller model). However, note that the pretrained model's output features—without the use of labels—already outperform the guessing of randomly initialized weights:

In [7]:
loss = training_report.validation_metrics['loss']
acc = training_report.validation_metrics['accuracy']
print(f'Final pretraining loss: {loss}')
print(f'Final pretraining KNN accuracy: {acc}')

Final pretraining loss: 907.89599609375
Final pretraining KNN accuracy: 36.95000076293945


Now you're ready to use Masterful's unsupervised pretraining API!